In [ ]:
!pip install transformers
!pip install nlpaug
!pip install sacremoses
!pip install sentencepiece

In [12]:
import numpy as np
import pandas as pd
from transformers import Trainer, AutoModelForSequenceClassification, ConvBertForSequenceClassification
import torch
import warnings
from google.colab import files
warnings.filterwarnings('ignore')




In [2]:
df_original=pd.read_csv("original_data1.csv")
df_original=df_original[['text','label']]
df_original.text.str.strip()

#80,10,10
df_train, df_test = np.split(df_original.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_original))])

print(df_train['label'].value_counts())
print(df_test['label'].value_counts())



df_only_1=df_train[df_train['label']==1]
df_only_2=df_train[df_train['label']==2]

0    3197
2     432
1      97
Name: label, dtype: int64
0    803
2     97
1     32
Name: label, dtype: int64


In [15]:
df_test.to_csv('original_test.csv')
files.download('original_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import nlpaug.augmenter.word as naw
import pandas as pd
import sentencepiece
from google.colab import files

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

def augment_data(df_train, df_only_1, df_only_2):

    back_translation_aug_en = naw.BackTranslationAug(
        from_model_name='facebook/wmt19-de-en',
        to_model_name='facebook/wmt19-en-de')

    back_translation_aug_fi = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-de-fi',
        to_model_name='Helsinki-NLP/opus-mt-fi-de')

    back_translation_aug_es = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-de-es',
        to_model_name='Helsinki-NLP/opus-mt-es-de')

    back_translation_aug_it = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-de-it',
        to_model_name='Helsinki-NLP/opus-mt-it-de')

    back_translation_aug_fr = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-de-fr',
        to_model_name='Helsinki-NLP/opus-mt-fr-de')

    back_translation_aug_pl = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-de-pl',
        to_model_name='Helsinki-NLP/opus-mt-pl-de')

    augment_model=[back_translation_aug_en,back_translation_aug_fi, back_translation_aug_es, back_translation_aug_it, back_translation_aug_fr, back_translation_aug_pl]


    printProgressBar(0, len(df_only_1['text']), prefix = 'Progress:', suffix = 'Complete', length = 50)
    i=0

    df_only_1=df_only_1.drop_duplicates(keep='first', ignore_index=True)
    temp=[]

    for s in df_only_1['text']:
        for aug in augment_model:
                final=aug.augment(s)
                temp.append({
                    'text':final[0],
                    'label':1
                })
        printProgressBar(i + 1,len(df_only_1['text']), prefix = 'Progress:', suffix = 'Complete', length = 50)
        i+=1

    df_temp=pd.DataFrame(temp)
    df_back = pd.concat([df_train, df_temp]).drop_duplicates(keep='first', ignore_index=True)
    df_back.to_csv("original+back(train+val)_only_1.csv")

    i=0
    printProgressBar(0, len(df_only_2['text']), prefix = 'Progress:', suffix = 'Complete', length = 50)

    df_only_2=df_only_2.drop_duplicates(keep='first', ignore_index=True)
    temp=[]

    for s in df_only_2['text']:
        for aug in augment_model:
                final=aug.augment(s)
                temp.append({
                    'text':final[0],
                    'label':2
                })
        printProgressBar(i + 1, len(df_only_2['text']), prefix = 'Progress:', suffix = 'Complete', length = 50)
        i+=1

    df_back = pd.concat([df_back, pd.DataFrame(temp)]).drop_duplicates(keep='first', ignore_index=True)

    df_back.to_csv("original+back(train+val).csv")
    
    files.download('original+back(train+val).csv')


This process took more than 7 hours, so pay attention when colab ask to you if you are using it or not.


In [5]:
augment_data(df_train=df_train, df_only_1=df_only_1, df_only_2=df_only_2)


Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [7]:
from google.colab import files
files.download('original+back(train+val).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>